In [1]:
import pandas as pd
import warnings
import tkinter as tk
warnings.filterwarnings('ignore')

In [29]:
cols ='user_id item_id rating timestamp'.split()
df=pd.read_csv('u.data',sep='\t',names=cols)
cols_movies=['item_id','title']+[str(i) for i in range(22)]
movie_titles = pd.read_csv('u.item',sep='|',names=cols_movies, encoding='latin-1')
movie_titles=movie_titles[['item_id','title']]
data= pd.merge(df,movie_titles,on='item_id')
rating=pd.DataFrame(data.groupby('title')['rating'].mean())
rating['count']=data['title'].value_counts()
pivot_df=data.pivot_table(index='user_id',columns ='title',values='rating')

app=tk.Tk()
app.title('Recomender System')
app.geometry('350x200')

tk.Label(app,text='Movie You Watched').place(x=25,y=25)
tk.Label(app,text='Recomended for you').place(x=25,y=100)
tk.Label(app,text='Also Try').place(x=25,y=125)

movie_var=tk.Variable(app)
tk.Entry(app,textvariable=movie_var,width =25).place(x=150,y=25)

recomend_var=tk.Variable(app)
recomend_var.set('Type Movie Name')
tk.Label(app,textvariable=recomend_var).place(x=150,y=100)

also_try_var=tk.Variable(app)
also_try_var.set('Type Movie Name')
tk.Label(app,textvariable=also_try_var).place(x=150,y=125)

def Find_recomendation():
    movie=movie_var.get().lower().strip()
    if movie:
        try:
            movie=movie_titles['title'][movie_titles['title'].apply(lambda x: movie in x.lower())].values[0]
            movie_var.set(movie)
        except IndexError:
            recomend_var.set('Movie Not Found')
            also_try_var.set('Movie Not Found')
        else:
            corr_df=pd.DataFrame(pivot_df.corrwith(pivot_df[movie]),columns =['Correlation'])
            corr_df.dropna(inplace = True)
            corr_df=corr_df.join(rating['count'])
            recomend_var.set(corr_df[(corr_df['count']>200)&(corr_df['Correlation']>0.4)].sort_values(by='Correlation',ascending=False).index[1])
            also_try_var.set((corr_df[corr_df['Correlation']>0.4].sort_values(by='Correlation',ascending=False).index[0]))

tk.Button(app,text='Find Suggestion',command=Find_recomendation, bg='#00f',fg='#fff').place(x=125,y=60)

app.mainloop()

Empire Strikes Back, The (1980)
Empire Strikes Back, The (1980)
